In [ ]:
import math
from collections import deque
from matplotlib import pyplot as plt

In [ ]:
Point = tuple[float, float]

# Create a board
class Board:
    xp, yp = [0, -1, 0, 1], [-1, 0, 1, 0]
    EPSILON = 1e-7

    def __init__(self, vertices: tuple[Point]):
        self.vertices = vertices
        assert len(self.vertices) > 2

        self.size = 0
        for Point in vertices:
            self.size = max(self.size, Point[0], Point[1])
        self.size = math.ceil(self.size)

        self.board = [[False for _ in range(self.size)]
                      for _ in range(self.size)]
        self.__calculate_board()

    def __crossed(self, cx: float, cy: float, nx: float, ny: float) -> bool:
        def __counter_clockwise(a: Point, b: Point, c: Point) -> float:
            return (a[0]-b[0])*c[1] + (b[0]-c[0])*a[1] + (c[0]-a[0])*b[1]

        def __different_half_plane(x: Point, y: Point, a: Point, b: Point) -> bool:
            return __counter_clockwise(x, a, b) * __counter_clockwise(y, a, b) < self.EPSILON

        def __intersected(c: Point, n: Point, a: Point, b: Point) -> bool:
            return __different_half_plane(c, n, a, b) and __different_half_plane(a, b, c, n)

        for i in range(len(self.vertices)):
            a, b = self.vertices[i-1], self.vertices[i]
            if __intersected((cx, cy), (nx, ny), a, b):
                return True
        return False

    def __calculate_board(self) -> None:
        queue = deque()
        queue.append((0, 0))
        while len(queue):
            cx, cy = queue.popleft()
            for xa, ya in zip(self.xp, self.yp):
                nx, ny = cx + xa, cy + ya
                if self.__crossed(cx + .5, cy + .5, nx + .5, ny + .5):
                    continue
                if self.board[nx][ny]:
                    continue
                self.board[nx][ny] = 1
                queue.append((nx, ny))
    
    def show(self) -> None:
        board = []
        for y in range(self.size):
            board.append([])
            for x in range(self.size):
                result = None
                match self.board[x][y]:
                    case 0:
                        result = 0, 0, 0, 0
                    case 1:
                        result = 90, 228, 165, 255
                    case 2:
                        result = 204, 201, 72, 255
                board[-1].append(result)
        plt.axis('off')
        plt.imshow(board, origin = "lower")

    def __repr__(self):
        s = ""
        for y in range(self.size-1, -1, -1):
            for x in range(self.size):
                s += 'X' if self.board[x][y] else '-'
            s += '\n'
        return s

BOARD1 = (.0, .0), (100.0, .0), (100.0, 50.0), (.0, 50.0)
BOARD2 = (.0, .0), (.0, 386.6025), (150.0, 236.6025), (150.0, 86.6025)
BOARD3 = (.0, .0), (450.0, .0), (450.0, 150.0), (150.0, 150.0), (300.0, 600.0), (.0, 450.0)


In [ ]:
# # Draw manually
# with open("figure1.txt", "w") as file:
#     print(Board(BOARD1), file=file)

# with open("figure2.txt", "w") as file:
#     print(Board(BOARD2), file=file)

# with open("figure3.txt", "w") as file:
#     print(Board(BOARD3), file=file)

Board(BOARD3).show()

In [ ]:
# Create a drone
class Drone:
    def __init__(self, capacity, max_battery, spray_range):
        self.capacity = capacity
        self.max_battery = max_battery

        self.spray_range = spray_range

        self.movement = [0, 0]
        self.position = [0, 0]
        self.spray = []
        self.last_spray = []

    def set_start_position(self, x, y):
        self.position = [x, y]
        self.update_spray()

    def set_movement(self, dx, dy):
        self.movement = [dx, dy]
        self.update_spray()

    def update_spray(self):
        try:
            mult = (self.spray_range / 2) / math.sqrt(self.movement[0] ** 2 + self.movement[1] ** 2)
            self.alt_movement = [self.movement[1], -self.movement[0]]
            self.last_spray = self.spray
            self.spray = [
                [self.position[0] - self.alt_movement[0] * mult, self.position[1] - self.alt_movement[1] * mult],
                [self.position[0] + self.alt_movement[0] * mult, self.position[1] + self.alt_movement[1] * mult]
            ]
            if not self.last_spray:
                self.last_spray = self.spray
        except ZeroDivisionError as e:
            pass

    def process(self):
        for d in range(2):
            self.position[d] += self.movement[d]
        self.update_spray()

    def draw(self):
        xs = [self.spray[0][0], self.spray[1][0]]
        ys = [self.spray[0][1], self.spray[1][1]]
        plt.plot(xs, ys, color = "#ccc948", linewidth = 3)


In [ ]:
drone = Drone(100, 100, 10)
drone.set_start_position(0, 0)
drone.set_movement(4, 3)

In [ ]:
def process_keyboard_event(event):
    key = event.key

    directions = {
        'w': [0, 1],
        'a': [-1, 0],
        's': [0, -1],
        'd': [1, 0],
    }

    if key in directions.keys():
        drone.set_movement(*directions[key])

    if key == ' ':
        drone.process()

In [ ]:
# drone.draw()
Board(BOARD3).show()
# plt.show()